# Initializer of the AARMP model in the following steps

1. #### Download and of NHD Data for Valley Bottom Models and creation of the two valley bottom models

	This checks for the existence of the data and then attempts download the NHD-HR data from Amazon S3 servers and create the remaining rasters variables from it. The some functions involved in this prep must be done within a python environment with access to the arcpy module.  

1. #### Download and inital prep of Landsat Data

	This checks for the existence of the final output and then, if it doesn't exist commences looking for the principal dataset and subsequent derivatives.

1. #### Georeference Landsat Data

	Begins creating the Valley Bottom raster based on the inputs prepared in step 1. 

1. #### Calculate Landsat NDWI and NDSI Indicies
    
    Calculation of Indicies derived from landsat bands (soil and water)

1. #### Initiation of the Landcover Analysis

	Beings creating landcover rasters for each QQuad of NAIP imagery. In the creation of each quarter quad, individual principals of vegetation indicies and gaussian blurs of NAIP bands, as well as resample and clipped landsats are prepared on demand. During this landcover classification, each quarter-quad the vegetation classes are isolated, density calculated, and a determination of potential riparian class created. Finally this qquad of potential riparian classes is clipped the valley bottom of that quarter quad.

Preliminary necessary variabels

In [8]:
from shapely.geometry import Point
from shapely.ops import linemerge
import rasterio as rio
import pandas as pd
import geopandas as gpd
import gdal, osr
import numpy as np
import os, shutil
import math
from rasterio.merge import merge as merge_tool

import Utilities as utils
import ValleyBottomRastersPrep
import VBET_ValleyBottomModel
import RSAC_RegressionModel
import getLandsatData
import buildLandsat

import logging

# Set to either INFO or DEBUG
logging.basicConfig(level=logging.INFO)

data_dir = os.path.abspath(r"M:\Data")
logging.info("Using main data directory '%s' for all non-NAIP files" % data_dir)

# Input vector files directory and files
vectors_dir = os.path.join(data_dir, "initial_model_inputs")

vb_classification_pnts = os.path.join(vectors_dir, "VM_TrainingData_20180619.shp")
watersheds_file = os.path.join(vectors_dir, "WBDHU4_Arizona.shp")
# Specify the Area of inteurest file and landsat path/row file (wrs2). An intersect will find all necessary path/rows
aoi_file = os.path.join(vectors_dir, "NAIP_AZ_Boundary.gpkg")
landsat_wrs2_file = os.path.join(vectors_dir, "wrs2_descending.shp")
naip_acqui_vector = os.path.join(vectors_dir, "NAIP2015_AcquisitionDates_Dissolve.gpkg")
# Find and download the NAIP acquisition dates for year of naip imagery. USDA has this,
# but it's hard to get. 2023 features in the layer with ~120 feature limit per download, 
# so had to chunk it into 100s for export then merge back together. Could also script. 
# Source: https://gis.apfo.usda.gov/arcgis/rest/services/NAIP_Status/NAIP_Image_Dates/MapServer/52
naip_acqui_raster = os.path.join(data_dir, "NAIP2015_AcquisitionDates_Dissolve.tif")
getLandsatData.rasterizeVector(naip_acqui_vector, naip_acqui_raster)

INFO:root:Using main data directory M:\Data for all non-NAIP files


### 1. Download and of NHD Data for Valley Bottom Models and creation of the two valley bottom models

In [ ]:
# Create NHD Data Directory - Hold Valley Bottom Data
nhd_dir = os.path.join(data_dir, "NHD")
watersheds_dir = os.path.join(nhd_dir, "Watersheds")

# ------------------------------------------------
# Download of necessary NHD raster data
getNHDData(nhd_dir)
  
# ------------------------------------------------
# VBET-based Valley Bottom Model, Currently the model used (as of 06/30/2018
large_slope_thresh = 2
medium_slope_thresh = 5
small_slope_thresh = 22

VBET_ValleyBottomModel.createVBETValleyBottom(nhd_dir, watersheds_dir, large_slope_thresh, medium_slope_thresh, small_slope_thresh, overwrite=False, cleanup=False)

# ------------------------------------------------
# RSAC Regression Model - better potential, currently poor in lower, flatter areas
RSAC_RegressionModel.valleyBottomRegression(watersheds_file, vb_classification_pnts, nhd_dir, watersheds_dir)

### 2. Download and inital prep of Landsat Data


In [ ]:
# Only DOY calc to determine which landsat scenes to download. Make sure it's the same as the NAIP year
year = 2015
# Maximum clouds over landsat scene for download on date
cloud_lim = 40

# Create landsat folder structure
landsat_dir = os.path.join(data_dir, "Landsat8")
utils.useDirectory(landsat_dir)
toa_dir = os.path.join(landsat_dir, "TOA_Corrected")
utils.useDirectory(toa_dir)


# ------------------------------------------------
# Download landsat and run TOA calculation and cloud mask
getLandsatData.getLandsatTOA(landsat_dir, naip_acqui_raster, aoi_file, landsat_wrs2_file, cloud_lim)

# ------------------------------------------------
# Using the NAIP DOY acquisition try to match for the landsat values at that location 
buildLandsat.createLandatDOYCompositeForAOI(naip_acqui_raster, data_dir, toa_dir, landsat_dir,
                                   landsat_toa_naipdate_merge, ls_nodata=landsat_nodata)

### 3. Georeference Landsat Data

The Landsat data is spatially offset from the NAIP data and needs to be georeferenced to overlap the best we can. The Easiest way to do this is in Arc, but you can only georeference 3 bands at a time. Workflow used was to georeference bands 1-3, then 2-6, 7, and then 8 and then use composite bands tool to merge them all back together. There's no easy way to script this without arcpy.

Provided is a sample set of control points in a text file that can be used

In [ ]:
georef_cntrl_points = os.path.abspath(vectors_dir, "ControlPoints_20180529.txt")
final_georef_landsat_composite = os.path.join(landsat_dir, "Landsat1to8_TOA_NAIPAcquiDate_merge_rectified.tif")

### 4. Calculate Landsat NDWI and NDSI Indicies

In [ ]:
import RasterCalculations as rs

ndwi_dir = os.path.join(data_dir, "NDWI")
ndsi_dir = os.path.join(data_dir, "NDSI")
rs.calculateLandsatIndicies(final_georef_landsat_composite, ndwi_outdir=ndwi_dir, ndsi_outdir=ndsi_dir)

### 5. Initiation of the Landcover Analysis

In [ ]:
import createRiparianClassifications